In [72]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, LeaveOneOut, cross_val_score
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import math

In [30]:
dataset = pd.read_csv('regressao_Q6.csv')

In [31]:
dataset.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,target
0,-0.351837,0.719462,0.862522,-1.131049,0.744903,0.532177,1.595572,0.607174,1.306696,0.450022,170.471696
1,-0.309565,0.786033,1.175365,-1.149287,0.318301,0.620987,1.470917,0.568968,1.334661,-0.267235,151.023999
2,-1.808729,-0.581660,-0.052200,-2.411602,0.171517,-1.174627,0.010317,-0.677372,0.841643,-1.186626,-268.606299
3,-0.386627,0.991724,0.234672,-0.431101,-0.160947,0.521185,1.957080,0.651099,0.933481,1.280705,180.737656
4,-0.879158,0.106556,0.775418,-1.046543,0.097267,-0.757026,1.050900,-0.262257,1.383563,-1.152509,-38.200673


In [32]:
dataset.shape

(1100, 11)

In [33]:
numeros_x = pd.DataFrame()
numeros_x = dataset.iloc[:,:-1]
print(numeros_x.head())
print(numeros_x.shape)

         x0        x1        x2        x3        x4        x5        x6  \
0 -0.351837  0.719462  0.862522 -1.131049  0.744903  0.532177  1.595572   
1 -0.309565  0.786033  1.175365 -1.149287  0.318301  0.620987  1.470917   
2 -1.808729 -0.581660 -0.052200 -2.411602  0.171517 -1.174627  0.010317   
3 -0.386627  0.991724  0.234672 -0.431101 -0.160947  0.521185  1.957080   
4 -0.879158  0.106556  0.775418 -1.046543  0.097267 -0.757026  1.050900   

         x7        x8        x9  
0  0.607174  1.306696  0.450022  
1  0.568968  1.334661 -0.267235  
2 -0.677372  0.841643 -1.186626  
3  0.651099  0.933481  1.280705  
4 -0.262257  1.383563 -1.152509  
(1100, 10)


In [34]:
numeros_y = pd.DataFrame()
numeros_y = dataset.iloc[:,-1:]
print(numeros_y.head())
print(numeros_y.shape)

       target
0  170.471696
1  151.023999
2 -268.606299
3  180.737656
4  -38.200673
(1100, 1)


In [36]:
numeros_x_values = numeros_x.values

In [37]:
numeros_y_values = numeros_y.values

In [38]:
numeros_x_values

array([[-0.35183672,  0.71946212,  0.86252193, ...,  0.60717393,
         1.30669552,  0.45002248],
       [-0.30956458,  0.78603289,  1.17536461, ...,  0.56896766,
         1.33466106, -0.26723532],
       [-1.80872907, -0.58165957, -0.05219979, ..., -0.6773723 ,
         0.84164275, -1.18662595],
       ...,
       [-1.15542263, -0.20606577,  0.63607072, ..., -0.35719131,
         1.45067207,  0.04963575],
       [-2.4141733 , -0.99367352,  0.56605337, ..., -1.21029804,
         1.11518776, -1.45009927],
       [-0.79442169,  0.66654512,  0.62789042, ...,  0.00957522,
         1.28425117, -0.05887021]])

In [74]:
def cross_validate_regr(cv,data_x,data_y,strength=1.0,model_type='Linear',random=None):
    scores_test = []
    scores_train = []
    y_pred_train = []
    y_pred_test = []
    mse_train = []
    mse_test = []
    mae_train = []
    mae_test = []
    
    for train_idx, test_idx in cv.split(data_x):
        X_train, X_test = data_x.iloc[train_idx], data_x.iloc[test_idx] 
        y_train, y_test = data_y.iloc[train_idx], data_y.iloc[test_idx]
        if model_type=='Linear':
            regr = LinearRegression(random_state=random)
        if model_type=='Ridge':
            regr = Ridge(alpha=strength,random_state=random)
        if model_type=='Lasso':
            regr = Lasso(alpha=strength,random_state=random)
        if model_type=='Elastic':
            regr = ElasticNet(alpha=strength, l1_ratio=0.5, random_state=random)
        regr.fit(X_train, y_train)
        y_pred_train = regr.predict(X_train)
        y_pred_test = regr.predict(X_test)
        scores_test.append(regr.score(X_test,y_test))
        scores_train.append(regr.score(X_train,y_train))
        mse_train.append(metrics.mean_squared_error(y_train,y_pred_train))
        mse_test.append(metrics.mean_squared_error(y_test,y_pred_test))
        mae_train.append(metrics.mean_absolute_error(y_train,y_pred_train))
        mae_test.append(metrics.mean_absolute_error(y_test,y_pred_test))
            
    print(model_type)
    print("TEST :")
    #print('R2 Test: ', scores_test)
    print('Media R2 Test Set: ',np.mean(scores_test))
    #print('MSE Test: ', mse_test)
    print("Mean MSE: ", np.mean(mse_test))
    print("Root Mean MSE: ", math.sqrt(np.mean(mse_test)))
    #print('MAE Test: ', mae_test)
    print("Mean MAE: ", np.mean(mae_test))
    print("Root Mean MAE: ", math.sqrt(np.mean(mae_test)))

    print("\n")

    print("TRAIN :")
    #print('R2 Train: ',scores_train)
    print('Media R2 Train Set: ',np.mean(scores_train))
    #print('MSE Train: ', mse_train)
    print("Mean MSE: ", np.mean(mse_train))
    print("Root Mean MSE: ", math.sqrt(np.mean(mse_train)))
    #print('MAE Train: ', mae_train)
    print("Mean MAE: ", np.mean(mae_train))
    print("Root Mean MAE: ", math.sqrt(np.mean(mae_train)))

    #print("Train: ", train_idx)
    #print("Test: ", test_idx)

In [65]:
#loo = LeaveOneOut()
cross_validate_regr(loo,numeros_x,numeros_y,strength=1.7,model_type='Ridge')

Ridge
TEST :
Media R2 Test Set:  0.0
Mean MSE:  767.3352439726468
Root Mean MSE:  27.700816666167928
Mean MAE:  22.05879761197035
Root Mean MAE:  4.696679424015477


TRAIN :
Media R2 Train Set:  0.995435919527455
Mean MSE:  752.5007813280808
Root Mean MSE:  27.431747690004748
Mean MAE:  21.84419229846353
Root Mean MAE:  4.673777091225418


In [66]:
regr = Ridge(alpha=1.7)
np.mean(cross_val_score(regr, numeros_x, numeros_y, cv=loo,scoring='mean_squared_error'))
#'accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'completeness_score', 
#'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 
#'homogeneity_score', 'mutual_info_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_squared_error', 
#'neg_mean_squared_log_error', 'neg_median_absolute_error', 'normalized_mutual_info_score', 'precision', 
#'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 
#'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'v_measure_score'

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

-767.3352439726468

In [67]:
for train_idx, test_idx in loo.split(numeros_x_values):
    X_train, X_test = numeros_x_values[train_idx], numeros_x_values[test_idx] 
    y_train, y_test = numeros_y_values[train_idx], numeros_y_values[test_idx]
    print("Train: ", train_idx)
    print("Test: ", test_idx)

Train:  [   1    2    3 ... 1097 1098 1099]
Test:  [0]
Train:  [   0    2    3 ... 1097 1098 1099]
Test:  [1]
Train:  [   0    1    3 ... 1097 1098 1099]
Test:  [2]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [3]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [4]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [5]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [6]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [7]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [8]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [9]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [10]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [11]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [12]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [13]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [14]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [15]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [16]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [17]
Tr

Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [250]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [251]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [252]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [253]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [254]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [255]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [256]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [257]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [258]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [259]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [260]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [261]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [262]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [263]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [264]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [265]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [266]
Train:  [   0    1    2 ... 109

Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [480]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [481]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [482]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [483]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [484]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [485]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [486]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [487]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [488]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [489]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [490]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [491]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [492]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [493]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [494]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [495]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [496]
Train:  [   0    1    2 ... 109

Test:  [729]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [730]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [731]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [732]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [733]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [734]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [735]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [736]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [737]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [738]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [739]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [740]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [741]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [742]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [743]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [744]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [745]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [746]
Train:  [   0    1

Test:  [979]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [980]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [981]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [982]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [983]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [984]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [985]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [986]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [987]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [988]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [989]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [990]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [991]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [992]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [993]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [994]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [995]
Train:  [   0    1    2 ... 1097 1098 1099]
Test:  [996]
Train:  [   0    1